In [17]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

#試取資料
file = open("./log/P2(1).pickle", "rb")
data = pickle.load(file)
file.close()


In [18]:
game_info = data["ml_2P"]["scene_info"]
game_command = data["ml_2P"]["command"]
print(game_info)
print(game_command)

[{'frame': 0, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (40, 240)}, {'frame': 1, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (45, 240)}, {'frame': 2, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (50, 240)}, {'frame': 3, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (55, 240)}, {'frame': 4, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (60, 240)}, {'frame': 5, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform_1P': (80, 420), 'platform_2P': (80, 50), 'blocker': (65, 240)}, {'frame': 6, 'status': 'GAME_ALIVE', 'ball': (98, 415), 'ball_speed': (0, 0), 'platform

In [19]:
 
for i in range(38, 88): #71開始18 17個 2-88
    path = "./log/P1(" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml_2P']['scene_info']
    game_command = game_command + data['ml_2P']['command']
    file.close()  
for i in range(45, 95): #78開始18 17個  9-95
    path = "./log/P2(" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    game_info = game_info + data['ml_2P']['scene_info']
    game_command = game_command + data['ml_2P']['command']
    file.close()
print(len(game_info))
print(len(game_command))

150584
150584


In [20]:
g = game_info[1]

feature = np.array([g["ball"][0], g["ball"][1], g["ball_speed"][0], g["ball_speed"][1], g["platform_2P"][0]+15,80])

print(feature)
print(game_command[1])
if game_command[1] == "NONE":
	game_command[1] = 0
elif game_command[1] == "MOVE_LEFT":
	game_command[1] = 1
else:
	game_command[1] = 2

[ 98 415   0   0  95  80]
None


In [21]:
for i in range(2, len(game_info) - 1):
    g = game_info[i]
    g_last = game_info[i-1]
    ball_x=g["ball"][0]
    ball_y=g["ball"][1]
    ball_vx=g['ball'][0] - g_last['ball'][0]
    ball_vy=g['ball'][1] - g_last['ball'][1]
    des_x = 80
    #if des_vy > 0:
    if ball_vy < 0:                                             #ball move up
                    if ball_vx < 0:                                         #ball move left
                        des_x = ball_x - (ball_y - 80)
                    else:                                                       #ball move right   
                        des_x = ball_x + (ball_y - 80)
    else:                                                           #ball move down  
                    if ball_vx < 0:                                         #ball move left
                        des_x = ball_x - (680 - (ball_y - 80))
                    else:                                                       #ball move right
                        des_x = ball_x + (680 - (ball_y - 80))
       
    while des_x > 200 or des_x < 0:
        if des_x > 200:
                des_x = 200 - (des_x - 200)
        else: 
                des_x = -des_x
    feature = np.vstack((feature, [g['ball'][0], g['ball'][1],ball_vx, ball_vy,g['platform_2P'][0]+15,des_x]))
    if game_command[i] == "NONE": game_command[i] = 0
    elif game_command[i] == "MOVE_LEFT": game_command[i] = 1
    else: game_command[i] = 2 
        
answer = np.array(game_command[1:-1])
print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 98 415   0   0  95  80]
 [ 98 415   0   0  95  43]
 [ 98 415   0   0  95  43]
 ...
 [176 412 -19  19 130 172]
 [157 431 -19  19 135 172]
 [155 450  -2  19 140 155]]
(150582, 6)
[2 2 2 ... 2 2 2]
(150582,)


# knn 

In [22]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9) #9
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=12)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=6) #n_jobs為平行運算的數量 #6
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('my_model_knn2.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    5.6s remaining:   11.3s
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    5.6s remaining:    5.6s
[Parallel(n_jobs=6)]: Done   4 out of   6 | elapsed:    5.7s remaining:    2.8s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    6.1s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    6.1s finished


In [23]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 1}
[[22296    22    51]
 [   35  8699     0]
 [   50     0 14022]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22369
           1       1.00      1.00      1.00      8734
           2       1.00      1.00      1.00     14072

    accuracy                           1.00     45175
   macro avg       1.00      1.00      1.00     45175
weighted avg       1.00      1.00      1.00     45175



# random forest 

In [24]:
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state = 7) #0.3 9(0)0會很多delay
#參數區間

forest = RandomForestClassifier(n_estimators=195, max_depth=130) #195 130
forest = forest.fit(x_train, y_train)
predicted = forest.predict(x_test)
file = open("mine_model_random2.pickle", "wb")
pickle.dump(forest, file)
file.close()
print("Accuracy score (validation): {0:.3f}".format(forest.score(x_test, y_test)))
print("Confusion Matrix for Raandom Forests:")
print(confusion_matrix(y_test, predicted))
print()
print("Classification Report for Random Forests")
print(classification_report(y_test, predicted))

Accuracy score (validation): 0.998
Confusion Matrix for Raandom Forests:
[[22348     6    48]
 [   27  8520     0]
 [   25     0 14201]]

Classification Report for Random Forests
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22402
           1       1.00      1.00      1.00      8547
           2       1.00      1.00      1.00     14226

    accuracy                           1.00     45175
   macro avg       1.00      1.00      1.00     45175
weighted avg       1.00      1.00      1.00     45175

